In [2]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
from keras.callbacks import History 
history = History()

In [3]:
train_path = "Train"
test_path = "Test"
valid_path = "Validate"

In [4]:
train_batches = ImageDataGenerator(rotation_range=0, width_shift_range=0.0, 
       height_shift_range=0.1, shear_range=0, zoom_range=0.1, 
       channel_shift_range=10.).flow_from_directory(train_path, target_size=(256,205), classes =['Normal','Glaucoma'], batch_size=32)
test_batches = ImageDataGenerator(rotation_range=0, width_shift_range=0.0, 
       height_shift_range=0.1, shear_range=0, zoom_range=0.1, 
       channel_shift_range=10.).flow_from_directory(test_path, target_size=(256,205), classes =['Normal','Glaucoma'], batch_size=32)

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [39]:
def plot_confusion_matrix(cm, classes, 
                          normalize=False,
                         title = 'Confusion Matrix',
                         cmap=plt.cm.Blues):
  
  plt.imshow(cm, interpolation = 'nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation = 45)
  plt.yticks(tick_marks, classes)
  
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1) [:, np.newaxis]
    print("Normalized Confusion Matrix")
  else:
    print("Confusion Matrix without normalization")
  
  print(cm)
  
  thresh = cm.max() / 2.
  
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j,i, cm[i,j],
            horizontalalignment ="center",
            color = "white" if cm[i,j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [40]:
vgg16_model = keras.applications.vgg16.VGG16()

In [41]:
type(vgg16_model)

tensorflow.python.keras.engine.functional.Functional

In [42]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [43]:
for layer in model.layers:
    layer.trainable = False

In [44]:
model.add(Dense(2, activation='softmax'))

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [46]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
history = model.fit_generator(train_batches, 
                    steps_per_epoch=20, 
                    validation_data= test_batches, 
                    validation_steps=20,
                    epochs=20,
                    verbose=2)

Instructions for updating:
Please use Model.fit, which supports generators.


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:
def plots(ims, figsize=(12,6), rows=5, interp=False, titles=None):
  if type(ims[0]) is np.ndarray:
    ims = np.array(ims).astype(np.uint8)
    if(ims.shape[-1] != 3):
      ims = ims.transpose ((0,2,3,1))
  f = plt.figure(figsize = figsize)
  cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows +1
  for i in range(len(ims)):
    sp = f.add_subplot (rows, cols, i+1)
    sp.axis('Off')
    if titles is not None:
      sp.set_title(titles[i], fontsize =10)
    plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
test_batches_fine = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, 
       height_shift_range=0.1, shear_range=0.15, zoom_range=0.1, 
       channel_shift_range=10., horizontal_flip=True).flow_from_directory(test_path, target_size=(224,224), classes =['Normal_ROI','Glaucoma_ROI'], batch_size=50)

In [ ]:
test_imgs, test_labels = next(test_batches_fine)

In [ ]:
x_labels = test_labels.argmax(axis=1)
print(x_labels)
plots(test_imgs, titles=test_labels)

In [ ]:
x_labels

In [ ]:
predictions = model.predict_generator(test_batches_fine, steps=1, verbose=0)

In [ ]:
y_labels = predictions.argmax(axis=1)
y_labels
#np.round(predictions[:,0])

In [ ]:
cm = confusion_matrix(x_labels, y_labels)

In [ ]:
cm_plot_labels = ['Normal','Glaucoma']

In [ ]:
plot_confusion_matrix(cm, cm_plot_labels, title ='Confusion Matrix')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()